In [56]:
import pandas as pd
import numpy as np

Here we read in data downloaded as CSV form from ``cdli_db``.

In [105]:
pubs = pd.read_csv("publications.csv", index_col = "id")
artifs_pubs = pd.read_csv("artifacts_publications.csv")
abbrev = pd.read_csv("abbreviations.csv")

C:\Users\Circle\AppData\Local\Temp/ipykernel_15992/1653827225.py:1: DtypeWarning: Columns (2,3,5,7,8,11,12,17,19,20,21,22,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  pubs = pd.read_csv("publications.csv", index_col = "id")
C:\Users\Circle\AppData\Local\Temp/ipykernel_15992/1653827225.py:2: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  artifs_pubs = pd.read_csv("artifacts_publications.csv")


# Step 1: Identify subsets of publications that may be curated.

A lot of fields have the form like "ATU 3, pl. 036, W 12139", where "ATU 3" is an assyriological abbreviation and what goes
after the comma is exact_reference.

We can pull out all the publications that has such "assyriological abbreviation".

So first we split the designation column by a comma and take what goes before the first comma. Name it "designation_first".
Then we pick all rows where there is a numeric character in "designation_first".

In [106]:
des_first = pubs["designation"].fillna("").str.split(",", n = 1, expand = True)[0]
pubs["designation_first"] = des_first
mergeable = pubs[pubs["designation_first"].str.match(".*\d.*")]


We assume these rows are formatted as "abbrev name + number", so extract the abbrev name.

Then we check if these names exist in the abbreviations table

Hopefully this can isolate some publications which may be merged.

In [107]:
mergeable_abbrevs = mergeable["designation_first"].str.split(" ", n = 1, expand = True)[0]
mergeable.loc[:, "designation_abbrev"] = mergeable_abbrevs

all_abbrevs = abbrev["abbreviation"].str.replace("\s\(aka.*\)", "").str.strip()
mergeable = mergeable[mergeable["designation_abbrev"].isin(all_abbrevs)]
mergeable.head()

C:\Users\Circle\AppData\Local\Temp/ipykernel_15992/335196345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergeable.loc[:, "designation_abbrev"] = mergeable_abbrevs
C:\Users\Circle\AppData\Local\Temp/ipykernel_15992/335196345.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  all_abbrevs = abbrev["abbreviation"].str.replace("\s\(aka.*\)", "").str.strip()


,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,school,title,volume,publication_history,series,oclc,accepted_by,accepted,designation_first,designation_abbrev
id,,,,,,,,,,,,,,,,,,,,,
468,CUSAS 35,Bartash2017,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Sumerian Administrative and Legal Documents ca...,NaN,NaN,NaN,NaN,820,1,CUSAS 35,CUSAS
472,ATU 3,Englund1993,1993,NaN,NaN,NaN,Die lexikalischen Listen der Archaischen Texte...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,ATU 3,ATU
483,"ATU 3, pl. 036, W 12139",1993-483,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,ATU 3,ATU
487,"ATU 3, pl. 080, W 13948",1993-487,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,ATU 3,ATU
488,"ATU 3, pl. 082, W 13982",1993-488,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,ATU 3,ATU


We can perform another round of filtering by removing designations with 1 entry. There's no point for merging them at this point 

In [108]:
def filter_func(df):
    return df.shape[0] > 1

mergeable = mergeable.groupby("designation_first").filter(filter_func)
mergeable.to_csv("before_merge.csv")
mergeable

,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,school,title,volume,publication_history,series,oclc,accepted_by,accepted,designation_first,designation_abbrev
id,,,,,,,,,,,,,,,,,,,,,
468,CUSAS 35,Bartash2017,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Sumerian Administrative and Legal Documents ca...,NaN,NaN,NaN,NaN,820,1,CUSAS 35,CUSAS
472,ATU 3,Englund1993,1993,NaN,NaN,NaN,Die lexikalischen Listen der Archaischen Texte...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,ATU 3,ATU
483,"ATU 3, pl. 036, W 12139",1993-483,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,ATU 3,ATU
487,"ATU 3, pl. 080, W 13948",1993-487,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,ATU 3,ATU
488,"ATU 3, pl. 082, W 13982",1993-488,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,ATU 3,ATU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000896,"RA 98, 148",1000896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,RA 98,RA
1000898,"RA 98, 148",1000898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,RA 98,RA
1000901,"RA 97, 171",1000901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,820,1,RA 97,RA


In [109]:
unchanged = pubs[~pubs.index.isin(mergeable.index)]
unchanged

,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,publisher,school,title,volume,publication_history,series,oclc,accepted_by,accepted,designation_first
id,,,,,,,,,,,,,,,,,,,,,
1,Bulla,Abusch1981,1981,7.0,Winona Lake,NaN,In Honor of Ernest R. Lacheman on His Seventy-...,NaN,NaN,NaN,...,Eisenbrauns,NaN,Notes on a Pair of Matching Texts: A Shepherd'...,1,NaN,SCCNH,NaN,820,1,Bulla
2,NaN,AbuschSchwemer2011,NaN,2.0,NaN,NaN,Corpus of Mesopotamian Anti-Witchcraft Rituals...,NaN,NaN,NaN,...,Brill: Leiden,NaN,NaN,8,NaN,Ancient Magic and Divination,NaN,820,1,
3,NaN,Alizadeh2008,NaN,2.0,NaN,NaN,Chogha Mish II: The Development of a Prehistor...,NaN,NaN,NaN,...,The Oriental Institute: Chicago,NaN,NaN,130,NaN,OIP,NaN,820,1,
4,NaN,Allred2006,NaN,15.0,NaN,NaN,Cooks and Kitchens: Centralized Food Productio...,NaN,NaN,NaN,...,NaN,Johns Hopkins University,NaN,NaN,NaN,NaN,NaN,820,1,
5,Girsu Labor,Allred2008,2008,7.0,NaN,NaN,On the Third Dynasty of Ur: Studies in Honor o...,NaN,NaN,NaN,...,The American Schools of Oriental Research,NaN,Labor Assignments from the City of Girsu,1,NaN,The Journal of Cuneiform Studies Supplemental ...,NaN,820,1,Girsu Labor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955152,"Kramer JAOS 060, 240",955152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,820,1,Kramer JAOS 060
990581,"Notizia, Palmiro, NABU 2018/001",990581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,820,1,Notizia
1000531,"RA 28, 124",1000531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,820,1,RA 28


# Step 2: Try to merge some of them.

This one goes through all unique values in the ``mergeable["designation_first"]`` column. However most have nothing to replace since they are just null.

In [110]:
cant_replace = []
for abbrev_name in mergeable["designation_first"].unique():

    # Has same name
    has_same_name = mergeable["designation_first"] == abbrev_name

    # Find all entries with that abbreviation name
    subset = mergeable[has_same_name]

    # Split into entries to be replaced (to_replace), and what they will be changed to (replace_with)
    to_replace, replace_with = subset[subset["book_title"].isna() & subset["title"].isna()], subset[~subset["book_title"].isna() | ~subset["title"].isna()]

    if replace_with.shape[0] < 1:
        to_replace, replace_with = subset.iloc[1:], subset.iloc[0:]
    
    # Treat what goes after the first comma in designation as exact_referece
    # Deal with designation with and without commas separately
    exact_reference = pd.Series(to_replace["designation"])
    has_comma = exact_reference.str.contains(",")
    if any(has_comma):
        exact_reference[has_comma] = exact_reference[has_comma].str.split(",", 1, True)[1]
    exact_reference[~has_comma] = ""

    # Fetch the id of the publications to be replaced
    ids_to_replace = to_replace.index

    # Fetch the id these publications will be changed to
    replace_id = replace_with.index[0]

    # Update the artifacts_publications table
    if not any(artifs_pubs["publication_id"].isin(ids_to_replace)):
        print("No corresponding entries in artifacts_publications.")
    else:
        artifs_pubs.loc[artifs_pubs["publication_id"].isin(ids_to_replace), "exact_reference"] = exact_reference
        artifs_pubs.loc[artifs_pubs["publication_id"].isin(ids_to_replace), "publication_id"] = replace_id
        artifs_pubs.head()
    
    # Update the mergeable table
    mergeable.loc[has_same_name, "designation"] = abbrev_name
    mergeable.drop(mergeable.index[mergeable.index.isin(ids_to_replace)], inplace = True)
    print("Merged")

Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged
Merged

In [111]:
mergeable.drop(columns = ["designation_first", "designation_abbrev"]).to_csv("merged_pubs.csv")

In [84]:
unchanged

,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,publisher,school,title,volume,publication_history,series,oclc,accepted_by,accepted,designation_first
id,,,,,,,,,,,,,,,,,,,,,
1,Bulla,Abusch1981,1981,7.0,Winona Lake,NaN,In Honor of Ernest R. Lacheman on His Seventy-...,NaN,NaN,NaN,...,Eisenbrauns,NaN,Notes on a Pair of Matching Texts: A Shepherd'...,1,NaN,SCCNH,NaN,820,1,Bulla
2,NaN,AbuschSchwemer2011,NaN,2.0,NaN,NaN,Corpus of Mesopotamian Anti-Witchcraft Rituals...,NaN,NaN,NaN,...,Brill: Leiden,NaN,NaN,8,NaN,Ancient Magic and Divination,NaN,820,1,
3,NaN,Alizadeh2008,NaN,2.0,NaN,NaN,Chogha Mish II: The Development of a Prehistor...,NaN,NaN,NaN,...,The Oriental Institute: Chicago,NaN,NaN,130,NaN,OIP,NaN,820,1,
4,NaN,Allred2006,NaN,15.0,NaN,NaN,Cooks and Kitchens: Centralized Food Productio...,NaN,NaN,NaN,...,NaN,Johns Hopkins University,NaN,NaN,NaN,NaN,NaN,820,1,
5,Girsu Labor,Allred2008,2008,7.0,NaN,NaN,On the Third Dynasty of Ur: Studies in Honor o...,NaN,NaN,NaN,...,The American Schools of Oriental Research,NaN,Labor Assignments from the City of Girsu,1,NaN,The Journal of Cuneiform Studies Supplemental ...,NaN,820,1,Girsu Labor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000901,"RA 97, 171",1000901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,820,1,RA 97
1013101,"RA 76, 011",1013101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,820,1,RA 76
1013102,"RA 76, 011",1013102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,820,1,RA 76
